In [12]:
import requests
import pandas as pd
from datetime import datetime
import os
from dotenv import load_dotenv
load_dotenv()
pwd = os.getenv('redshift_pwd')
#Verify it worked
if pwd is not None:
    print('It worked')

It worked


In [13]:
url = 'http://dataservice.accuweather.com/currentconditions/v1/topcities/150?apikey='
api_key = 'behGGPCV9pYyEfwrUqPfIjb4axaqJZ0a'

full_url = url + api_key

response = requests.get(full_url).json()

# Inicializamos listas para almacenar los datos obtenidos de la API

nombres_paises = []
nombres_ciudades = []
descripcion = []
temperatura = []
fecha_medicion = []
datetimes = []

# Iteramos sobre cada elemento del JSON obtenido
for data in response:
    # Extraemos los datos relevantes
    nombre_pais = data['Country']['LocalizedName']
    nombre_ciudad = data['LocalizedName']
    weather_text = data['WeatherText']
    temperature = data['Temperature']['Metric']['Value']
    timestamp = data['LocalObservationDateTime']
    datetime_insertion = datetime.now()

    # Agrega los datos a las listas+
    nombres_paises.append(nombre_pais)
    nombres_ciudades.append(nombre_ciudad)
    descripcion.append(weather_text)
    temperatura.append(temperature)
    fecha_medicion.append(timestamp)

    # Creamos DataFrame
    df = pd.DataFrame({
        "Nombre de País": nombres_paises,
        "Nombre de Ciudad": nombres_ciudades,
        "Descripción":descripcion,
        "Temperatura (C)": temperatura,
        "Fecha de Medición": fecha_medicion,
        "Fecha de Inserción": datetime_insertion
    })

df



,Nombre de País,Nombre de Ciudad,Descripción,Temperatura (C),Fecha de Medición,Fecha de Inserción
0,Bangladesh,Dhaka,Clear,27.8,2024-03-04T18:32:00+06:00,2024-03-04 09:46:40.116036
1,Democratic Republic of the Congo,Kinshasa,Partly sunny,31.7,2024-03-04T13:21:00+01:00,2024-03-04 09:46:40.116036
2,Chile,Santiago,Sunny,16.9,2024-03-04T09:35:00-03:00,2024-03-04 09:46:40.116036
3,China,Beijing,Overcast,3.9,2024-03-04T20:33:00+08:00,2024-03-04 09:46:40.116036
4,Colombia,Bogota,Mostly cloudy,12.6,2024-03-04T07:35:00-05:00,2024-03-04 09:46:40.116036
...,...,...,...,...,...,...
145,Timor-Leste,Dili,Cloudy,28.4,2024-03-04T21:35:00+09:00,2024-03-04 09:46:40.116036
146,Indonesia,Banda Aceh,Thunderstorm,27.2,2024-03-04T19:35:00+07:00,2024-03-04 09:46:40.116036
147,Cyprus,Nicosia,Partly sunny,18.7,2024-03-04T14:32:00+02:00,2024-03-04 09:46:40.116036
148,Gibraltar,Gibraltar,Cloudy,17.8,2024-03-04T13:31:00+01:00,2024-03-04 09:46:40.116036


In [14]:
# Creación de conexión con RedShift
import psycopg2
url = 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
database = 'data-engineer-database'
user = 'marianolicera3_coderhouse'

try:
    conn = psycopg2.connect(
        host=url,
        dbname=database,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Conectado a Redshift con éxito!")
    
except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)

Conectado a Redshift con éxito!


In [15]:
#Crear la tabla si no existe

with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS marianolicera3_coderhouse.weather
        (
            nombre_pais VARCHAR(255),
            nombre_ciudad VARCHAR(255),
            descripcion VARCHAR(50),
            temperatura FLOAT,
            fecha_medicion DATETIME,
            fecha_insercion DATETIME 
        )
    """)
    conn.commit()
#nombre de país VARCHAR(200)
# nombre de ciudad VARCHAR(200)
# descripcion VARCHAR(50)
# temperatura FLOAT
# fecha de medicion DATETIME
# fecha de insercion DATETIME

In [16]:
#Vaciar la tabla para evitar duplicados o inconsistencias
with conn.cursor() as cur:
    cur.execute("Truncate table weather")
    count = cur.rowcount

In [17]:
#Insertando los datos en Redsfhift
from psycopg2.extras import execute_values
with conn.cursor() as cur:
    execute_values(
        cur,
        '''
        INSERT INTO weather (nombre_pais, nombre_ciudad, descripcion, temperatura, fecha_medicion, fecha_insercion)
        VALUES %s
        ''',
        [tuple(row) for row in df.values],
        page_size=len(df)
    )
    conn.commit()

In [18]:
#consultando la tabla
cur = conn.cursor()
cur.execute("SELECT * FROM weather LIMIT 5")
results = cur.fetchall()
results

[('Bangladesh',
  'Dhaka',
  'Clear',
  27.8,
  datetime.datetime(2024, 3, 4, 18, 32),
  datetime.datetime(2024, 3, 4, 9, 46, 40, 116036)),
 ('Democratic Republic of the Congo',
  'Kinshasa',
  'Partly sunny',
  31.7,
  datetime.datetime(2024, 3, 4, 13, 21),
  datetime.datetime(2024, 3, 4, 9, 46, 40, 116036)),
 ('Chile',
  'Santiago',
  'Sunny',
  16.9,
  datetime.datetime(2024, 3, 4, 9, 35),
  datetime.datetime(2024, 3, 4, 9, 46, 40, 116036)),
 ('China',
  'Beijing',
  'Overcast',
  3.9,
  datetime.datetime(2024, 3, 4, 20, 33),
  datetime.datetime(2024, 3, 4, 9, 46, 40, 116036)),
 ('Colombia',
  'Bogota',
  'Mostly cloudy',
  12.6,
  datetime.datetime(2024, 3, 4, 7, 35),
  datetime.datetime(2024, 3, 4, 9, 46, 40, 116036))]